In [ ]:
import requests
from requests import post, get
from dotenv import load_dotenv
import os
import base64
import json

In [ ]:
load_dotenv()

In [ ]:
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

print(client_id, client_secret)

# Get token

In [ ]:
import requests
import base64
import json

# Function to obtain the access token from Spotify API
def get_token():
    # Spotify client ID and secret
    client_id = "ENTER YOUR CLIENT ID"
    client_secret = "ENTER YOUR CLIENT SECRET"

    # Concatenate client ID and secret with a colon
    auth_string = client_id + ":" + client_secret
    # Encode the concatenated string to base64
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
    # Spotify token endpoint URL
    url = "https://accounts.spotify.com/api/token"
    # Headers required for obtaining the token
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }

    data = {"grant_type": "client_credentials"}
    result = requests.post(url, headers=headers, data=data)
    # If the request is successful (status code 200), extract the token from the response
    if result.status_code == 200:
        # Parse the JSON response
        json_result = result.json()
        token = json_result["access_token"]
        return token
    else:
        print("Error:", result.status_code)
        return None

# Function to create the authorization header for subsequent API requests
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

# Function to search for an artist on Spotify
def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"

    query_url = url + query
    result = get(query_url, headers=headers)
    # Parse the JSON response
    json_result = json.loads(result.content)
    print(json_result)

token = get_token()
if token:
    print("token: ", token)
    search_for_artist(token, "Drake")
else:
    print("Failed to obtain token.")


## Top 10 songs of an artist

In [ ]:
# import spotify_app as st
import requests
import base64
import json

# Function to obtain the access token from Spotify API
def get_token():
    # Spotify client ID and secret
    client_id = "ENTER YOUR CLIENT ID"
    client_secret = "ENTER YOUR CLIENT SECRET"

    # Concatenate client ID and secret with a colon
    auth_string = client_id + ":" + client_secret
    # Encode the concatenated string to base64
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
    # Spotify token endpoint URL
    url = "https://accounts.spotify.com/api/token"
    # Headers required for obtaining the token
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }

    data = {"grant_type": "client_credentials"}
    result = requests.post(url, headers=headers, data=data)
    # If the request is successful (status code 200), extract the token from the response
    if result.status_code == 200:
        # Parse the JSON response
        json_result = result.json()
        token = json_result["access_token"]
        return token
    else:
        print("Failed to obtain token.")
        return None

# Function to create the authorization header for subsequent API requests
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"

    query_url = url + query
    result = get(query_url, headers=headers)

    json_result = json.loads(result.content)["artists"]["items"]
    if len(json_result) == 0:
        print("No artist with this name exists...")
        return None
    
    return json_result[0]

def get_songs_by_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)["tracks"]
    return json_result


token = get_token()
result = search_for_artist(token, "Playboi Carti")
artist_id = result["id"]
songs = get_songs_by_artist(token, artist_id)
# print(songs)

for idx, song in enumerate(songs):
    print(f"{idx + 1}.{song['name']}")


In [ ]:
# token = get_token()
# result = search_for_artist(token, "Justin Bieber")
# artist_id = result["id"]
# songs = get_songs_by_artist(token, artist_id)
# # print(songs)

# for idx, song in enumerate(songs):
#     print(f"{idx + 1}.{song['name']}")

## Gradio

In [ ]:
# !pip install gradio --upgrade

In [ ]:
import gradio as gr
import requests
import base64
import json

# Function to obtain the access token from Spotify API
def get_token():
    client_id = "ENTER YOUR CLIENT ID"
    client_secret = "ENTER YOUR CLIENT SECRET"

    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }

    data = {"grant_type": "client_credentials"}
    result = requests.post(url, headers=headers, data=data)
    if result.status_code == 200:
        json_result = result.json()
        token = json_result["access_token"]
        return token
    else:
        print("Failed to obtain token.")
        return None

# Function to create the authorization header for subsequent API requests
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"
    query_url = url + query
    result = requests.get(query_url, headers=headers)
    json_result = result.json()["artists"]["items"]
    if len(json_result) == 0:
        return None
    return json_result[0]

def get_top_songs(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"
    headers = get_auth_header(token)
    result = requests.get(url, headers=headers)
    json_result = result.json()["tracks"]
    return json_result

# # Function to format the top songs into a table for Gradio output
# def format_songs_table(songs):
#     headers = ["Song", "Album", "Popularity"]
#     rows = []
#     for song in songs:
#         rows.append([song["name"], song["album"]["name"], song["popularity"]])
#     return [headers] + rows

def search_and_display(artist_name):
    token = get_token()
    if not token:
        return "Failed to obtain token. Please try again later."

    artist = search_for_artist(token, artist_name)
    if not artist:
        return "Artist not found."

    songs = get_songs_by_artist(token, artist['id'])
    if not songs:
        return "No top songs found for this artist."

    songs.sort(key=lambda x:x['popularity'], reverse=True)
    # Extracting attributes separately
    formatted_songs = "\n".join([f"{idx + 1}. {song['name']} - {song['popularity']}" for idx, song in enumerate(songs[:10])])
    return formatted_songs


from gradio.themes.base import Base
from gradio.themes.utils import colors, fonts, sizes
from typing import Iterable

class CustomTheme(Base):
    def __init__(
        self,
        *,
        primary_hue: colors.Color | str = colors.slate,
        secondary_hue: colors.Color | str = colors.lime,
        neutral_hue: colors.Color | str = colors.neutral,
        spacing_size: sizes.Size | str = sizes.spacing_md,
        radius_size: sizes.Size | str = sizes.radius_md,
        text_size: sizes.Size | str = sizes.text_lg,
        font: fonts.Font
        | str
        | Iterable[fonts.Font | str] = (
            fonts.GoogleFont("Quicksand"),
            "ui-sans-serif",
            "sans-serif",
        ),
        font_mono: fonts.Font
        | str
        | Iterable[fonts.Font | str] = (
            fonts.GoogleFont("IBM Plex Mono"),
            "ui-monospace",
            "monospace",
        ),
    ):
        super().__init__(
            primary_hue=primary_hue,
            secondary_hue=secondary_hue,
            neutral_hue=neutral_hue,
            spacing_size=spacing_size,
            radius_size=radius_size,
            text_size=text_size,
            font=font,
            font_mono=font_mono,
        )
        super().set(
            body_background_fill="black",  # Set background color to black
            # button_primary_background_fill=secondary_hue,  # Set button background to green
            # button_primary_text_color="darkgreen",  # Set button text color to white
            slider_color="white",  # Set slider color to white
            block_title_text_color="black",  # Set block title text color to white
            block_shadow="none", 
             # Remove block shadow
        )

spotify_theme = CustomTheme()

custom_css = """
h1, .gr-description, .gr-title, p {
    color: white !important;
}

input[type="text"], textarea {
    background-color: darkgrey !important;
    color: black !important;
    font-weight: bold !important; /* Make the text bold */
}

#component-11, .lg.primary {
    background-color: #1DB954 !important; /* Change button background color to Spotify green */
    color: white !important; /* Change button text color to white */
}

span.svelte-1gfkn6j  {
    font-weight: bold !important; /* Make the label text bold */
    color: black !important;  /* Ensure the label text color is white */
"""

spotify_logo_url = "https://developer.spotify.com/images/guidelines/design/icon3@2x.png"

gr.Interface(
    search_and_display,
    inputs=gr.Textbox(label="Enter Artist Name"),
    outputs=gr.Textbox(label="Top 10 Songs"),
    # title="Spotify Top 10 Songs by Artist",
    title=f'<img src="{spotify_logo_url}" alt="Spotify Logo" style="height:50px; vertical-align:middle;"> Spotify Top 10 Songs by Artist',
    description="Enter the artist's name to display their top 10 songs.",
    allow_flagging= "never",
    theme = spotify_theme,
    css=custom_css  # Apply custom theme
).launch()


# gr.Interface(
#     search_and_display,
#     inputs=gr.Textbox(label="Enter Artist Name"),
#     outputs=gr.Textbox(label="Top 10 Songs"),
#     title="Spotify Top 10 Songs by Artist",
#     description="Enter the artist's name to display their top 10 songs.",
#     allow_flagging= "never"
# ).launch()


# EDA

In [ ]:
import requests
import base64
import json

# Function to obtain the access token from Spotify API
def get_token():
    client_id = "ENTER YOUR CLIENT ID"
    client_secret = "ENTER YOUR CLIENT SECRET"

    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }

    data = {"grant_type": "client_credentials"}
    result = requests.post(url, headers=headers, data=data)
    if result.status_code == 200:
        json_result = result.json()
        token = json_result["access_token"]
        return token
    else:
        print("Failed to obtain token.")
        return None

# Function to create the authorization header for subsequent API requests
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"
    query_url = url + query
    result = requests.get(query_url, headers=headers)
    json_result = result.json()["artists"]["items"]
    if len(json_result) == 0:
        return None
    return json_result[0]

def get_songs_by_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"
    headers = get_auth_header(token)
    result = requests.get(url, headers=headers)
    json_result = result.json()["tracks"]
    return json_result

def get_audio_features(token, track_id):
    url = f"https://api.spotify.com/v1/audio-features/{track_id}"
    headers = get_auth_header(token)
    result = requests.get(url, headers=headers)
    audio_features = result.json()
    return audio_features

# Function to display artist's top tracks and their audio features
def display_top_tracks_with_features(artist_name):
    token = get_token()
    if not token:
        return "Failed to obtain token. Please try again later."

    artist = search_for_artist(token, artist_name)
    if not artist:
        return "Artist not found."

    songs = get_songs_by_artist(token, artist['id'])
    if not songs:
        return "No top songs found for this artist."

    # Print artist's top tracks
    print("Top Tracks:")
    for idx, song in enumerate(songs):
        print(f"{idx + 1}. {song['name']}")

    # Retrieve and display audio features for each track
    print("\nAudio Features:")
    for idx, song in enumerate(songs):
        audio_features = get_audio_features(token, song['id'])
        relevant_features = {
            'danceability': audio_features['danceability'],
            'energy': audio_features['energy'],
            'key': audio_features['key'],
            'loudness': audio_features['loudness'],
            'mode': audio_features['mode'],
            'speechiness': audio_features['speechiness'],
            'acousticness': audio_features['acousticness'],
            'instrumentalness': audio_features['instrumentalness'],
            'liveness': audio_features['liveness'],
            'valence': audio_features['valence'],
            'tempo': audio_features['tempo']
        }
        print(f"{idx + 1}. {song['name']}: {relevant_features}")

# Example usage:
display_top_tracks_with_features("Drake")


## Gradio

In [ ]:
import requests
import base64
import json
import pandas as pd

# Function to obtain the access token from Spotify API
def get_token():
    client_id = "ENTER YOUR CLIENT ID"
    client_secret = "ENTER YOUR CLIENT SECRET"

    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }

    data = {"grant_type": "client_credentials"}
    result = requests.post(url, headers=headers, data=data)
    if result.status_code == 200:
        json_result = result.json()
        token = json_result["access_token"]
        return token
    else:
        print("Failed to obtain token.")
        return None

# Function to create the authorization header for subsequent API requests
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

#Function to search for the artist
def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"
    query_url = url + query
    result = requests.get(query_url, headers=headers)
    json_result = result.json()["artists"]["items"]
    if len(json_result) == 0:
        return None
    return json_result[0]

#Function to get the songs by that artist
def get_songs_by_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"
    headers = get_auth_header(token)
    result = requests.get(url, headers=headers)
    json_result = result.json()["tracks"]
    return json_result

#Function to get the audio features of the artist's tracks
def get_audio_features(token, track_id):
    url = f"https://api.spotify.com/v1/audio-features/{track_id}"
    headers = get_auth_header(token)
    result = requests.get(url, headers=headers)
    audio_features = result.json()
    return audio_features

# Function to sort tracks by a chosen feature
def sort_tracks_by_feature(artist_name, feature):
    token = get_token()
    if not token:
        return "Failed to obtain token. Please try again later."

    artist = search_for_artist(token, artist_name)
    if not artist:
        return "Artist not found."

    songs = get_songs_by_artist(token, artist['id'])
    if not songs:
        return "No top songs found for this artist."

    tracks_with_features = []
    for song in songs:
        audio_features = get_audio_features(token, song['id'])
        if feature in audio_features:
            tracks_with_features.append({
                'name': song['name'],
                'Feature Score': audio_features[feature]
            })

    sorted_tracks = sorted(tracks_with_features, key=lambda x: x['Feature Score'], reverse=True)

    # output = f"Top tracks for {artist_name} sorted by {feature}:\n"
    # output += "Audio Features:\n"
    # for idx, track in enumerate(sorted_tracks):
    #     output += f"{idx + 1}. {track['name']}: {track['feature_score']}\n"

    return sorted_tracks

# Front end: Create a gradio interface 
import gradio as gr

def sort_tracks_by_feature_interface(artist_name, feature):
    output = sort_tracks_by_feature(artist_name, feature)
    if isinstance(output, str):
        return None, output # Hide the DataFrame and return the error message

    # tracks = [{'Track': track['name'], 'Feature Score': track['feature_score']} for track in output]
    df = pd.DataFrame(output)
    return df, None
    # return sort_tracks_by_feature(artist_name, feature)

from gradio.themes.base import Base
from gradio.themes.utils import colors, fonts, sizes
from typing import Iterable

class CustomTheme(Base):
    def __init__(
        self,
        *,
        primary_hue: colors.Color | str = colors.slate,
        secondary_hue: colors.Color | str = colors.lime,
        neutral_hue: colors.Color | str = colors.neutral,
        spacing_size: sizes.Size | str = sizes.spacing_md,
        radius_size: sizes.Size | str = sizes.radius_md,
        text_size: sizes.Size | str = sizes.text_lg,
        font: fonts.Font
        | str
        | Iterable[fonts.Font | str] = (
            fonts.GoogleFont("Quicksand"),
            "ui-sans-serif",
            "sans-serif",
        ),
        font_mono: fonts.Font
        | str
        | Iterable[fonts.Font | str] = (
            fonts.GoogleFont("Quicksand"),
            "ui-sans-serif",
            "sans-serif",
        ),
    ):
        super().__init__(
            primary_hue=primary_hue,
            secondary_hue=secondary_hue,
            neutral_hue=neutral_hue,
            spacing_size=spacing_size,
            radius_size=radius_size,
            text_size=text_size,
            font=font,
            font_mono=font_mono,
        )
        super().set(
            body_background_fill="black",  # Set background color to black
            # button_primary_background_fill=secondary_hue,  # Set button background to green
            # button_primary_text_color="darkgreen",  # Set button text color to white
            slider_color="white",  # Set slider color to white
            block_title_text_color="black",  # Set block title text color to white
            block_shadow="none", 
             # Remove block shadow
        )

spotify_theme = CustomTheme()

#Custom css used to modify some features that were not able to be covered earlier in the gradio interface
custom_css = """
h1, .gr-description, .gr-title, p {
    color: white !important;
}

input[type="text"], textarea {
    background-color: darkgrey !important;
    color: black !important;
    font-weight: bold !important; /* Make the text bold */
    
}

#component-11, .lg.primary {
    background-color: #1DB954 !important; /* Change button background color to Spotify green */
    color: white !important; /* Change button text color to white */
}

span.svelte-1gfkn6j  {
    font-weight: bold !important; /* Make the label text bold */
    color: black !important;  /* Ensure the label text color is white */

.output-column {
    display: flex;
    flex-direction: column;
    gap: 2rem;
}

.output-dataframe {
    font-family: 'Quicksand', 'ui-sans-serif', 'sans-serif' !important;
    font-weight: bold !important;
    background-color: #333333 !important;
    color: white !important;
    padding: 1rem;
    border-radius: 0.5rem;
    overflow: auto;
    max-height: 300px;
}


"""

spotify_logo_url = "https://developer.spotify.com/images/guidelines/design/icon3@2x.png"

#Gradio interface final code and for it to run
gr.Interface(
    sort_tracks_by_feature_interface,
    inputs=[
        gr.Textbox(label="Enter Artist Name", placeholder="🔍 Search..."),
        gr.Dropdown(choices=["energy", "danceability", "valence", "acousticness", "instrumentalness", "liveness", "speechiness"], label="Select Audio Feature", value="energy")
    ],
    outputs=[
        gr.Dataframe(headers=["Track", "Feature Score"], label="Sorted Tracks"),
        gr.Textbox(label="Error Message", visible=False)
    ],
    title=f'<img src="{spotify_logo_url}" alt="Spotify Logo" style="height:50px; vertical-align:middle;"> Hottest Tracks by Audio Feature',
    description="Enter an artist's name and select an audio feature to sort their top tracks by that feature.",
    allow_flagging="never",
    theme=spotify_theme,
    css=custom_css  # Apply custom theme
).launch()

           
   